In [165]:
# !pip install thefuzz

In [4]:
import pandas as pd
import sys
import numpy as np
import re
from thefuzz import process

In [2]:
bundesliga = pd.read_csv('1-bundesliga.csv')
eredivisie = pd.read_csv('eredivisie.csv')
liganos = pd.read_csv('liga-nos.csv')
ligue1 = pd.read_csv('ligue-1.csv')
premier = pd.read_csv('premier-league.csv')
premierliga = pd.read_csv('premier-liga.csv')
liga = pd.read_csv('primera-division.csv')
seriea = pd.read_csv('serie-a.csv')

In [3]:
df = pd.concat([bundesliga, eredivisie, liganos, ligue1, premier, premierliga, liga, seriea], ignore_index = True)
df.head()

,club_name,player_name,age,position,club_involved_name,fee,transfer_movement,transfer_period,fee_cleaned,league_name,year,season,country
0,1.FC Saarbrücken,Thomas Stratos,25.0,Defensive Midfield,Hamburger SV,€150Th.,in,Summer,0.15,1 Bundesliga,1992,1992/1993,Germany
1,1.FC Saarbrücken,Matthias Lust,22.0,Left Midfield,Waldh. Mannheim,free transfer,in,Summer,0.00,1 Bundesliga,1992,1992/1993,Germany
2,1.FC Saarbrücken,Henning Bürger,22.0,Left Midfield,FC Schalke 04,free transfer,in,Summer,0.00,1 Bundesliga,1992,1992/1993,Germany
3,1.FC Saarbrücken,Michael Hennig,20.0,midfield,FC Berlin,free transfer,in,Summer,0.00,1 Bundesliga,1992,1992/1993,Germany
4,1.FC Saarbrücken,Malik Badji,24.0,defence,CS Sedan,?,in,Summer,NaN,1 Bundesliga,1992,1992/1993,Germany


In [4]:
len(df)

153171

In [5]:
# Remove duplicated rows
df = df.drop_duplicates()
len(df)

153146

In [7]:
# CLEANING

# Remove 'Career break', 'Unknown', 'Ban', 'Retired' and 'Without Club'
df = df[df['club_involved_name'] != 'Career break']
df = df[df['club_involved_name'] != 'Unknown']
df = df[df['club_involved_name'] != 'Ban']
df = df[df['club_involved_name'] != 'Retired']
df = df[df['club_involved_name'] != 'Without Club']

# Remove df[df['fee'] == '0'] - Only one observation
df = df[df['fee'] != '0']

# Remove transfers involving seocond teams/youth sectors
remove = ['II', 'U19', 'U20', ' B', ' C', '-D', 'Youth', 'U21', 'U17', 'U23', 'U18', 
          'Sub-23', ' 2', 'Res.', 'Juve Next Gen', 'Arsenal-2 Tula', 'FShM Torpedo 19', 
          'US Palermo Yout', 'Cremonese Giov.', 'Amkar-Junior', 'Blackburn Acad.', 'Reggina Primaver', 
          'Real Oviedo You', 'Tom-2 Tomsk', 'DYuSSh Spartak', 'CA Osasuna Prom', 'Y19', 
          'Yth.', 'ACR Messina You', 'Akademia KSS', 'Valladolid Prom', 'Southampton Aca', 
          'Siena Junior', 'Leeds Reserves', 'Anzhi-Yunior', 'Barça Atlètic', 'Akademia Ufa']
remove_escaped = [re.escape(x) for x in remove]
pattern = r'\b(?:' + '|'.join(remove_escaped) + r')(?!\w)'
df = df[~df['club_involved_name'].str.contains(pattern, regex = True)]
        
len(df)

134279

In [8]:
# Add 'id' column
df['id'] = range(0, len(df))

In [9]:
#Reorder columns
new_order = ['id', 'club_name', 'player_name', 'age', 'position', 
             'club_involved_name', 'fee', 'transfer_movement', 
             'transfer_period', 'fee_cleaned', 'league_name', 
             'year', 'season', 'country']

df = df[new_order]

In [10]:
df.reset_index(drop = True, inplace = True)
df.head()

,id,club_name,player_name,age,position,club_involved_name,fee,transfer_movement,transfer_period,fee_cleaned,league_name,year,season,country
0,0,1.FC Saarbrücken,Thomas Stratos,25.0,Defensive Midfield,Hamburger SV,€150Th.,in,Summer,0.15,1 Bundesliga,1992,1992/1993,Germany
1,1,1.FC Saarbrücken,Matthias Lust,22.0,Left Midfield,Waldh. Mannheim,free transfer,in,Summer,0.00,1 Bundesliga,1992,1992/1993,Germany
2,2,1.FC Saarbrücken,Henning Bürger,22.0,Left Midfield,FC Schalke 04,free transfer,in,Summer,0.00,1 Bundesliga,1992,1992/1993,Germany
3,3,1.FC Saarbrücken,Michael Hennig,20.0,midfield,FC Berlin,free transfer,in,Summer,0.00,1 Bundesliga,1992,1992/1993,Germany
4,4,1.FC Saarbrücken,Malik Badji,24.0,defence,CS Sedan,?,in,Summer,NaN,1 Bundesliga,1992,1992/1993,Germany


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 134279 entries, 0 to 134278
Data columns (total 14 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   id                  134279 non-null  int64  
 1   club_name           134279 non-null  object 
 2   player_name         134279 non-null  object 
 3   age                 134245 non-null  float64
 4   position            134279 non-null  object 
 5   club_involved_name  134279 non-null  object 
 6   fee                 134184 non-null  object 
 7   transfer_movement   134279 non-null  object 
 8   transfer_period     134279 non-null  object 
 9   fee_cleaned         57480 non-null   float64
 10  league_name         134279 non-null  object 
 11  year                134279 non-null  int64  
 12  season              134279 non-null  object 
 13  country             134279 non-null  object 
dtypes: float64(2), int64(2), object(10)
memory usage: 14.3+ MB


In [176]:
def match_name(name, standard_names):
    return process.extractOne(name, standard_names)

df['transfer_id'] = None

datain = df[df['transfer_movement'] == 'in']
dataout = df[df['transfer_movement'] == 'out']

k = 0

CLEAR_LINE = '\033[K'

for i, transfer_in in datain.iterrows():
    index_in = transfer_in['id']
    if transfer_in['transfer_id'] is not None:
        continue
    search = dataout[dataout['season'] == transfer_in['season']]
    search = search[search['transfer_period'] == transfer_in['transfer_period']]
    search = search[search['player_name'] == transfer_in['player_name']]
    search = search[search['age'] == transfer_in['age']]
    search = search[search['position'] == transfer_in['position']]
    search = search[search['fee'] == transfer_in['fee']]
    search = search[search['transfer_id'].isnull()]
    if len(search) > 1:
        teams = list(search['club_involved_name'])
        target = match_name(transfer_in['club_name'], teams)[0]
        search = search[search['club_involved_name'] == target]
    for j, transfer_out in search.iterrows():
        index_out = transfer_out['id']
        datain.loc[datain['id'] == index_in, 'transfer_id'] = k
        dataout.loc[dataout['id'] == index_out, 'transfer_id'] = k
        k += 1

    message = f"Iteration {i}, {np.round(i*100/len(df), 2)}%"
    sys.stdout.write('\r' + CLEAR_LINE + message)
    sys.stdout.flush()

Iteration 134273, 100.0%0


In [204]:
in_notnull = datain[datain['transfer_id'].notnull()].copy()
in_notnull['country2'] = None
out_notnull = dataout[dataout['transfer_id'].notnull()].copy()

for index, row_in in in_notnull.iterrows():
    id = row_in['transfer_id']
    row_out = out_notnull[out_notnull['transfer_id'] == id]
    team_out = row_out['club_name'].values[0]
    contryout = row_out['country'].values[0]
    in_notnull.loc[index, 'club_involved_name'] = team_out
    in_notnull.loc[index, 'country2'] = contryout
    
in_notnull.head() 

,id,club_name,player_name,age,position,club_involved_name,fee,transfer_movement,transfer_period,fee_cleaned,league_name,year,season,country,transfer_id,country2
0,0,1.FC Saarbrücken,Thomas Stratos,25.0,Defensive Midfield,Hamburger SV,€150Th.,in,Summer,0.15,1 Bundesliga,1992,1992/1993,Germany,0,Germany
2,2,1.FC Saarbrücken,Henning Bürger,22.0,Left Midfield,FC Schalke 04,free transfer,in,Summer,0.00,1 Bundesliga,1992,1992/1993,Germany,1,Germany
8,8,1.FC Saarbrücken,Arno Glesius,26.0,Centre-Forward,Karlsruher SC,?,in,Summer,NaN,1 Bundesliga,1992,1992/1993,Germany,2,Germany
9,9,1.FC Saarbrücken,Wolfram Wuttke,30.0,Second Striker,RCD Espanyol Barcelona,?,in,Summer,NaN,1 Bundesliga,1992,1992/1993,Germany,3,Spain
23,23,SG Wattenscheid 09,Marek Lesniak,28.0,Centre-Forward,Bayer 04 Leverkusen,free transfer,in,Summer,0.00,1 Bundesliga,1992,1992/1993,Germany,4,Germany


In [208]:
network_data = in_notnull[['club_name', 'country', 'club_involved_name', 'country2', 'fee', 'fee_cleaned', 'season', 'transfer_period']].copy()
network_data.rename(columns = {'club_name': 'target', 'country': 'target_country', 'club_involved_name': 'source', 'country2': 'source_country'}, inplace = True)
network_data.reset_index(drop = True, inplace = True)
network_data.head()

,target,target_country,source,source_country,fee,fee_cleaned,season,transfer_period
0,1.FC Saarbrücken,Germany,Hamburger SV,Germany,€150Th.,0.15,1992/1993,Summer
1,1.FC Saarbrücken,Germany,FC Schalke 04,Germany,free transfer,0.00,1992/1993,Summer
2,1.FC Saarbrücken,Germany,Karlsruher SC,Germany,?,NaN,1992/1993,Summer
3,1.FC Saarbrücken,Germany,RCD Espanyol Barcelona,Spain,?,NaN,1992/1993,Summer
4,SG Wattenscheid 09,Germany,Bayer 04 Leverkusen,Germany,free transfer,0.00,1992/1993,Summer


In [209]:
len(network_data)

25721

In [6]:
network_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 25721 entries, 0 to 25720
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   target           25721 non-null  object 
 1   target_country   25721 non-null  object 
 2   source           25721 non-null  object 
 3   source_country   25721 non-null  object 
 4   fee              25721 non-null  object 
 5   fee_cleaned      13147 non-null  float64
 6   season           25721 non-null  object 
 7   transfer_period  25721 non-null  object 
dtypes: float64(1), object(7)
memory usage: 1.8+ MB
